In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [3]:
def dividir_rango_fechas(desde, hasta):
    """Divide un rango de fechas en segmentos de hasta un año."""
    inicio = []
    final = []
    fecha_actual = desde
    while fecha_actual < hasta:
        siguiente = min(fecha_actual + timedelta(days=365), hasta)
        inicio.append(fecha_actual)
        final.append(siguiente)
        fecha_actual = siguiente + timedelta(days=1)
    return inicio, final


In [ ]:
principales_variables = pd.read_csv('principalesVariables.csv')
idvariables = principales_variables['idVariable'].tolist() 
fecha_inicio = datetime(1996,1,1)
fecha_final = datetime.now()
inicio, final = dividir_rango_fechas(fecha_inicio, fecha_final)
empty_df = pd.DataFrame()
df_dicc={}
fechas = pd.date_range(start=fecha_inicio, end=fecha_final, freq='D')
registros_bcra = pd.DataFrame({'fecha': fechas})

In [ ]:
for v in idvariables:
    for i, f in zip(inicio, final):
        url = f"https://api.bcra.gob.ar/estadisticas/v2.0/datosvariable/{v}/{i.strftime('%Y-%m-%d')}/{f.strftime('%Y-%m-%d')}"
        try:
            response = requests.get(url, verify=False)

            # Convertir la respuesta JSON a un objeto de Python
            data = response.json()
        
            results = data.get("results")
            if results:
                print(f'Cargando datos {i} a {f}...')
                # Crear un DataFrame temporal con los resultados
                temp_df = pd.DataFrame(results)
                # Concatenar el DataFrame temporal con el acumulador
                empty_df = pd.concat([empty_df, temp_df])
            else:
                print(f'No hay datos en rango {i} a {f}')

        except KeyError as e:
            print(f"Error procesando los datos para el rango {i} - {f}: {e}")

    df_dicc[v] = empty_df
    empty_df = empty_df.iloc[0:0]
    print('='*40)
    print(f'Variable {v} cargada.')
    print('='*40)
    print('='*40)

    

In [ ]:
for clave, df in df_dicc.items():
    df['fecha'] = pd.to_datetime(df['fecha'])
    # Combinar los datos basados en la columna 'fecha'
    registros_bcra = registros_bcra.merge(
        df[['fecha', 'valor']],
        on='fecha',
        how='left',
        suffixes=('', f'_{clave}')  # Evita conflictos en los nombres
    )
    # Renombrar la columna 'valor' resultante al nombre de la clave
    registros_bcra.rename(columns={'valor': clave}, inplace=True)

# Mostrar el resultado
print(registros_bcra)

ValueError: You are trying to merge on datetime64[ns] and object columns for key 'fecha'. If you wish to proceed you should use pd.concat